In [2]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from datetime import datetime
import sys

In [5]:
# Get the current working directory (which should be 'notebooks/')
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Add the 'data' directory to the path
sys.path.insert(0, os.path.join(parent_dir, 'data'))

# Now you can import your module
from measurments import get_measurments


# 1. Connection Settings

In [9]:
# Load .env file
load_dotenv()

# Get the EC2 tracking server host from the environment variable
EC2_TRACKING_SERVER_HOST = os.getenv('EC2_TRACKING_SERVER_HOST')
EC2_ENDPOINT = f"http://{EC2_TRACKING_SERVER_HOST}:8000"

# Parameters for the RDS PostgreSQL instance
PG_HOST = os.getenv('PG_HOST')
PG_PORT = os.getenv('PG_PORT')
PG_DATABASE = os.getenv('PG_DATABASE')
PG_USER = os.getenv('PG_USER')
PG_PASSWORD = os.getenv('PG_PASSWORD')

# Create the MySQL database connection string
db_url = f'postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}'

In [10]:
def ingest_rds(df, table_name, mode):
    engine = create_engine(db_url)
    connection = engine.connect()
    df.to_sql(table_name, engine, if_exists=mode, index=False)
    connection.close()

In [28]:
df = pd.DataFrame({
    'model_name': ['xgboost-8features-hpt', 'xgboost-8features-hpt-guster3'],
    'retrained_date': [datetime(2023,11,5,23,0,14) , datetime(2023,11,5,23,2,31)]  # Gets today's date
})

In [29]:
ingest_rds(df, 'table_update_rewa', 'append')

In [13]:
def select_from_rds(query):
    engine = create_engine(db_url)
    connection = engine.connect()
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [30]:
select_from_rds('select * from table_update_rewa')

,model_name,retrained_date
0,xgboost-8features-hpt,2023-11-05 23:00:14
1,xgboost-8features-hpt-guster3,2023-11-05 23:02:31
2,xgboost-8features-hpt,2023-11-05 23:00:14
3,xgboost-8features-hpt-guster3,2023-11-05 23:02:31


In [24]:
df_m = select_from_rds('select * from measurments_rewa')
df_m['Time'].max()

Timestamp('2023-11-07 20:30:00')

In [28]:
df = select_from_rds(query='select * from model_versions')

# 2. Kuznica setup

In [7]:
df_measurments_kuznica = get_measurments(station = 'kuznica',past_days=1020)

/Users/krystianpietrzak/Documents/ML/WindPRO/data/measurments.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df_i])


In [11]:
ingest_rds(df_measurments_kuznica, 'measurments_kuznica', 'replace')

In [17]:
df_measurments = select_from_rds('select * from measurments_kuznica')
df_measurments

,Time,WindSpeed,WindGust,Update
0,2021-01-23 00:00:00,7.19,7.97,2023-11-09 18:51:40.594281
1,2021-01-23 00:10:00,7.19,7.97,2023-11-09 18:51:40.594281
2,2021-01-23 00:20:00,7.19,7.97,2023-11-09 18:51:40.594281
3,2021-01-23 00:30:00,7.19,7.97,2023-11-09 18:51:40.594281
4,2021-01-23 00:40:00,7.19,7.97,2023-11-09 18:51:40.594281
...,...,...,...,...
143667,2023-11-09 01:30:00,10.50,13.80,2023-11-09 18:51:40.594281
143668,2023-11-09 01:40:00,9.91,11.86,2023-11-09 18:51:40.594281
143669,2023-11-09 01:50:00,9.91,13.22,2023-11-09 18:51:40.594281
143670,2023-11-09 02:00:00,8.55,11.86,2023-11-09 18:51:40.594281


In [18]:
df_forecast = select_from_rds('select * from forecast')
df_forecast

,Time,Month,Hour,WindForecast,GustForecast,WindDirForecast,Temperature,Precipitation,Cloudcover,Update
0,2021-01-01 02:00:00,1,2,7.0,12.00,191.0,2.00,0.0,97.0,2023-10-03
1,2021-01-01 04:00:00,1,4,7.0,11.00,184.0,2.00,0.0,26.0,2023-10-03
2,2021-01-01 06:00:00,1,6,8.0,12.00,171.0,2.00,0.0,100.0,2023-10-03
3,2021-01-01 08:00:00,1,8,9.0,13.00,166.0,2.00,0.0,100.0,2023-10-03
4,2021-01-01 10:00:00,1,10,9.0,14.00,164.0,2.00,0.0,100.0,2023-10-03
...,...,...,...,...,...,...,...,...,...,...
12138,2023-10-30 14:00:00,10,14,9.2,17.20,228.5,13.50,0.0,100.0,2023-10-30
12139,2023-10-30 16:00:00,10,16,6.7,11.45,205.0,11.75,0.0,100.0,2023-10-30
12140,2023-10-30 18:00:00,10,18,8.0,18.00,196.5,10.75,0.0,98.0,2023-10-30
12141,2023-10-30 20:00:00,10,20,7.0,17.10,199.0,10.45,0.0,100.0,2023-10-30


In [19]:
# Set the 'Time' column as the index
df_measurments.set_index('Time', inplace=True)

# Resample the data with a two-hour interval and apply mean aggregation
df_measurments = df_measurments.resample('2H').mean()

df_measurments.reset_index(inplace=True)

df = pd.merge(left=df_forecast, right=df_measurments, on='Time', how='inner')

df.dropna(inplace=True)
df

,Time,Month,Hour,WindForecast,GustForecast,WindDirForecast,Temperature,Precipitation,Cloudcover,Update_x,WindSpeed,WindGust,Update_y
3,2021-01-23 00:00:00,1,0,5.00,10.00,208.0,4.00,0.00,100.0,2023-10-03,7.368333,8.438333,2023-11-09 18:51:40.594281216
4,2021-01-23 02:00:00,1,2,7.00,17.00,237.0,3.00,0.00,100.0,2023-10-03,7.208333,8.697500,2023-11-09 18:51:40.594281216
5,2021-01-23 04:00:00,1,4,9.00,19.00,243.0,3.00,0.00,74.0,2023-10-03,3.920000,5.670833,2023-11-09 18:51:40.594281216
6,2021-01-23 06:00:00,1,6,8.00,16.00,237.0,2.00,0.00,99.0,2023-10-03,6.010000,8.342500,2023-11-09 18:51:40.594281216
7,2021-01-23 08:00:00,1,8,4.00,7.00,206.0,2.00,0.00,98.0,2023-10-03,2.363333,3.547500,2023-11-09 18:51:40.594281216
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11875,2023-11-08 14:00:00,11,14,11.55,19.65,226.5,8.70,0.95,100.0,2023-11-08,12.150000,16.295833,2023-11-09 18:51:40.594281216
11876,2023-11-08 16:00:00,11,16,13.95,23.60,239.0,8.95,0.15,97.5,2023-11-08,12.115000,15.372500,2023-11-09 18:51:40.594281216
11877,2023-11-08 18:00:00,11,18,14.35,24.30,246.5,8.85,0.05,98.5,2023-11-08,14.627500,18.205000,2023-11-09 18:51:40.594281216
11878,2023-11-08 20:00:00,11,20,14.65,24.50,245.5,8.75,0.00,98.0,2023-11-08,14.158333,17.607500,2023-11-09 18:51:40.594281216
